In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = sns.load_dataset('tips')

In [ ]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


#**Problem Statement: Predict what is the time if anyone is visiting for Lunch or Dinner**

In [ ]:
df.time.unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [ ]:
df.isnull().sum()

,0
total_bill,0
tip,0
sex,0
smoker,0
day,0
time,0
size,0


In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['time'] = encoder.fit_transform(df['time'])
df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,0,2
1,10.34,1.66,Male,No,Sun,0,3
2,21.01,3.50,Male,No,Sun,0,3
3,23.68,3.31,Male,No,Sun,0,2
4,24.59,3.61,Female,No,Sun,0,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,0,3
240,27.18,2.00,Female,Yes,Sat,0,2
241,22.67,2.00,Male,Yes,Sat,0,2
242,17.82,1.75,Male,No,Sat,0,2


In [ ]:
df['time'].unique()

array([0, 1])

In [ ]:
X = df.drop('time', axis=1)
y = df['time']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=1)

In [ ]:
X_test.head()

,total_bill,tip,sex,smoker,day,size
67,3.07,1.00,Female,Yes,Sat,1
243,18.78,3.00,Female,No,Thur,2
206,26.59,3.41,Male,Yes,Sat,3
122,14.26,2.50,Male,No,Thur,2
89,21.16,3.00,Male,No,Thur,2


In [ ]:
from sklearn.impute import SimpleImputer #for missing value
from sklearn.preprocessing import OneHotEncoder #for encoding
from sklearn.preprocessing import StandardScaler #for scaling


from sklearn.pipeline import Pipeline #a sequence of data transformer
from sklearn.compose import ColumnTransformer #Groups all the pipeline steps for each of the columns

In [ ]:
cat_cols = ["sex", "smoker", "day"]
num_cols = ["total_bill", "tip", "size"]

In [ ]:
#feature engineering automation using pipeline and columntransformer

num_pipeline = Pipeline(steps = [('imputation', SimpleImputer(strategy = "median")),
                                ('scaling', StandardScaler())])
cat_pipeline = Pipeline(steps = [('imputation', SimpleImputer(strategy = "most_frequent")),
                                ('encoding', OneHotEncoder())])

In [ ]:
preprocessor = ColumnTransformer([("num_pipeline", num_pipeline, num_cols),
                  ("cat_pipeline", cat_pipeline, cat_cols)])

In [ ]:
preprocessor

ColumnTransformer(transformers=[('num_pipeline',
                                 Pipeline(steps=[('imputation',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaling',
                                                  StandardScaler())]),
                                 ['total_bill', 'tip', 'size']),
                                ('cat_pipeline',
                                 Pipeline(steps=[('imputation',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoding',
                                                  OneHotEncoder())]),
                                 ['sex', 'smoker', 'day'])])

In [ ]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [ ]:
#now data is ready, lets build the multiple models

from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [ ]:
models = {"support vector classifier": SVC(),
         "DT Classifier": DecisionTreeClassifier(),
         "Log reg": LogisticRegression(),
         "RF": RandomForestClassifier()}
models

{'support vector classifier': SVC(),
 'DT Classifier': DecisionTreeClassifier(),
 'Log reg': LogisticRegression(),
 'RF': RandomForestClassifier()}

In [ ]:
for i in range(len(models)):
    print(list(models.values())[i])

SVC()
DecisionTreeClassifier()
LogisticRegression()
RandomForestClassifier()


In [ ]:
from sklearn.metrics import accuracy_score

def model_train_eval(X_train, X_test, y_train, y_test, models):
    evaluation = {}
    for i in range(len(models)):
        model = list(models.values())[i]
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        model_score = accuracy_score(y_test, y_pred)
        evaluation[list(models.keys())[i]] = model_score
    return evaluation

In [ ]:
model_train_eval(X_train, X_test, y_train, y_test, models)

{'support vector classifier': 0.9183673469387755,
 'DT Classifier': 0.8979591836734694,
 'Log reg': 0.9183673469387755,
 'RF': 0.8979591836734694}

#**Random Forest Model:**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf

RandomForestClassifier()

In [ ]:
X_train, X_test

(array([[-0.28611937, -1.47443803, -0.57766863, ...,  0.        ,
          1.        ,  0.        ],
        [ 0.02695905, -0.71612531,  1.47042924, ...,  0.        ,
          1.        ,  0.        ],
        [ 1.3716196 ,  1.19880579,  1.47042924, ...,  0.        ,
          1.        ,  0.        ],
        ...,
        [-0.23206267,  0.43283335, -0.57766863, ...,  0.        ,
          0.        ,  1.        ],
        [-1.06543688, -1.29060464, -0.57766863, ...,  1.        ,
          0.        ,  0.        ],
        [-0.29287646,  0.1034652 ,  0.44638031, ...,  1.        ,
          0.        ,  0.        ]]),
 array([[-1.85376383, -1.48209775, -1.60171757,  1.        ,  0.        ,
          0.        ,  1.        ,  0.        ,  1.        ,  0.        ,
          0.        ],
        [-0.08453291,  0.04984713, -0.57766863,  1.        ,  0.        ,
          1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          1.        ],
        [ 0.79501474,  0.363895

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
params = {'max_depth': [1, 2, 3, 4, 5, 10, None],
         'n_estimators': [30, 40, 50, 100, 200, 300],
         'criterion': ['gini', 'entropy']}

params

{'max_depth': [1, 2, 3, 4, 5, 10, None],
 'n_estimators': [30, 40, 50, 100, 200, 300],
 'criterion': ['gini', 'entropy']}

In [ ]:
clf = RandomizedSearchCV(rf, param_distributions=params, cv = 5, verbose=3, scoring = 'accuracy', n_iter=10)

In [ ]:
clf

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [1, 2, 3, 4, 5, 10, None],
                                        'n_estimators': [30, 40, 50, 100, 200,
                                                         300]},
                   scoring='accuracy', verbose=3)

In [ ]:
clf.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=entropy, max_depth=None, n_estimators=50;, score=0.923 total time=   0.1s
[CV 2/5] END criterion=entropy, max_depth=None, n_estimators=50;, score=0.974 total time=   0.1s
[CV 3/5] END criterion=entropy, max_depth=None, n_estimators=50;, score=1.000 total time=   0.1s
[CV 4/5] END criterion=entropy, max_depth=None, n_estimators=50;, score=0.974 total time=   0.1s
[CV 5/5] END criterion=entropy, max_depth=None, n_estimators=50;, score=0.974 total time=   0.1s
[CV 1/5] END criterion=entropy, max_depth=2, n_estimators=100;, score=0.923 total time=   0.3s
[CV 2/5] END criterion=entropy, max_depth=2, n_estimators=100;, score=0.974 total time=   0.3s
[CV 3/5] END criterion=entropy, max_depth=2, n_estimators=100;, score=1.000 total time=   0.3s
[CV 4/5] END criterion=entropy, max_depth=2, n_estimators=100;, score=1.000 total time=   0.3s
[CV 5/5] END criterion=entropy, max_depth=2, n_estimators=100;, score=1.00

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [1, 2, 3, 4, 5, 10, None],
                                        'n_estimators': [30, 40, 50, 100, 200,
                                                         300]},
                   scoring='accuracy', verbose=3)

In [ ]:
clf.best_params_

{'n_estimators': 100, 'max_depth': 2, 'criterion': 'entropy'}

In [ ]:
clf.best_score_

np.float64(0.9794871794871796)

In [ ]:
final_model = clf.best_estimator_
final_model.predict(X_test)

array([0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0])

In [ ]:
#internal homework

#use total_bill as target variable and make it a regressor problem
#automate fe, taraing, testing and also random forest regressor with MLR and SVR model